In [91]:
import pandas as pd
import numpy as np

In [92]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [93]:
import asyncpraw

In [94]:
reddit = await asyncpraw.Reddit(
    client_id=os.environ["REDDIT_CLIENT_ID"],
    client_secret=os.environ["REDDIT_CLIENT_SECRET"],
    # password=os.environ.get("REDDIT_PASSWORD"),
    user_agent=os.environ["USER_AGENT"]
    #, username=os.envirton.get("REDDIT_USERNAME")
).__aenter__()

# reddit.read_only = True|False (if using OAuth, default=False if OAuth is used)

### Ratelimits

In [95]:
print(reddit.auth.limits['used']) # if connected to OAuth

None


In [96]:
print(reddit.auth.limits['remaining']) # if connected to OAuth

None


In [97]:
print(reddit.auth.limits['reset_timestamp']) # if connected to OAuth

None


### Data retrieval

In [98]:
wsb = await reddit.subreddit("wallstreetbets")

In [103]:
submission_df = []

In [ ]:
async for item in wsb.:
    print(item)

BadRequest: received 400 HTTP response

In [104]:
submissions = wsb.new(limit=2) # Must recreate for each async iteration
async for submission in submissions:
    await submission._fetch() # Necessary line
    submission_comments = []
    async for comment in submission.comments:
        submission_comments.append(comment.body)
    submission_df.append([submission.title, submission.url, submission.selftext, submission_comments])

In [105]:
pd.DataFrame(submission_df)

,0,1,2,3
0,No clear evidence linking Tylenol during pregn...,https://www.bmj.com/content/391/bmj-2025-088141,My position: 100 KVUE JAN 16’26 20 Calls,[\n**User Report**| | | |\n:--|:--|:--|:--\n**...
1,Bought $50k worth of QQQ calls. No idea what I...,https://i.redd.it/7n16wbivdd0g1.jpeg,March 2026 exp. Is this going to print hard???...,[\n**User Report**| | | |\n:--|:--|:--|:--\n**...


In [89]:
await reddit.close()

In [ ]:
subreddit = await reddit.subreddit("wallstreetbets+finance+investing")
async for submission in subreddit.sticky():
    print(submission)

In [113]:
subreddit = await reddit.subreddit("wallstreetbets")
thread = await subreddit.sticky(2)

In [ ]:
thread.title
thread.comments